# Session 1: React Hooks – useState & useEffect
## Comprehensive Training Guide for Fresh Engineering Graduates

---

## Table of Contents
1. [Introduction & Motivation](#introduction--motivation)
2. [Functional vs Class Components](#functional-vs-class-components)
3. [useState Hook - Deep Dive](#usestate-hook---deep-dive)
4. [useEffect Hook - Complete Lifecycle](#useeffect-hook---complete-lifecycle)
5. [Common Mistakes & Solutions](#common-mistakes--solutions)
6. [Practical Examples & Patterns](#practical-examples--patterns)
7. [Exercises & Assessment](#exercises--assessment)

---

## Introduction & Motivation

### What are React Hooks?

React Hooks are **special functions** that allow you to "hook into" React features from functional components. Before Hooks (React < 16.8), you could only use state and lifecycle features in **class components**. Hooks changed everything.

**Key Facts:**
- Introduced in React 16.8 (February 2019)
- Allow functional components to have state and side effects
- Enable better code reuse through custom hooks
- Make components simpler and more readable
- Are the modern standard (class components still work but are legacy)

### Why Hooks Were Created

Class components had several pain points:

| Problem | Impact | Hooks Solution |
|---------|--------|----------------|
| **State logic scattered** | Code split across `constructor`, `componentDidMount`, `componentDidUpdate`, `componentWillUnmount` | All state logic in one place with `useEffect` |
| **Hard to reuse logic** | Had to use render props or HOCs (complex patterns) | Custom hooks - simple function reuse |
| **Large components** | Unrelated logic mixed together | Split concerns naturally |
| **Difficult to understand** | Lifecycle methods force a time-based structure, not data-based | Effects group related logic |
| **"this" binding complexity** | Every method needs binding or arrow functions | No "this" needed in hooks |

---

## Functional vs Class Components

### Class Components (Legacy Pattern)

```javascript
// OLD WAY - Class Component
class Counter extends React.Component {
  constructor(props) {
    super(props);
    this.state = { count: 0 };
  }

  componentDidMount() {
    console.log('Component mounted');
    document.title = `Count: ${this.state.count}`;
  }

  componentDidUpdate(prevProps, prevState) {
    if (prevState.count !== this.state.count) {
      console.log('Count changed from', prevState.count, 'to', this.state.count);
      document.title = `Count: ${this.state.count}`;
    }
  }

  componentWillUnmount() {
    console.log('Component unmounting, cleaning up...');
  }

  render() {
    return (
      <div>
        <p>Count: {this.state.count}</p>
        <button onClick={() => this.setState({ count: this.state.count + 1 })}>
          Increment
        </button>
      </div>
    );
  }
}
```

**Problems with this approach:**
1. Logic is scattered across three lifecycle methods
2. Hard to see which effects are related
3. State updates are object merges (can cause issues)
4. "this" binding required for methods

### Functional Components with Hooks (Modern Pattern)

```javascript
// NEW WAY - Functional Component with Hooks
function Counter() {
  const [count, setCount] = useState(0);

  useEffect(() => {
    console.log('Component mounted or count changed');
    document.title = `Count: ${count}`;

    return () => {
      console.log('Cleanup before next effect or unmount');
    };
  }, [count]); // Dependency array

  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={() => setCount(count + 1)}>
        Increment
      </button>
    </div>
  );
}
```

**Advantages:**
1. ✅ All related logic in one `useEffect`
2. ✅ Simpler, more readable code
3. ✅ No "this" keyword needed
4. ✅ Better code organization
5. ✅ Easier to test

---

## useState Hook - Deep Dive

### Basic Syntax

```javascript
const [value, setValue] = useState(initialValue);
```

**Breaking it down:**
- `useState` - Hook function from React
- `value` - Current state value
- `setValue` - Function to update state
- `initialValue` - Initial state value (can be a function for expensive computations)

### Example 1: Simple State (Primitive Type)

```javascript
import { useState } from 'react';

function SimpleCounter() {
  // useState returns an array: [currentValue, updateFunction]
  const [count, setCount] = useState(0);
  const [name, setName] = useState('');
  const [isVisible, setIsVisible] = useState(true);

  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={() => setCount(count + 1)}>Increment</button>
      
      <input
        value={name}
        onChange={(e) => setName(e.target.value)}
        placeholder="Enter name"
      />
      
      <button onClick={() => setIsVisible(!isVisible)}>Toggle</button>
      {isVisible && <p>Visible content</p>}
    </div>
  );
}
```

### Example 2: Object State (Immutability Pattern)

**❌ WRONG - Mutating state directly:**
```javascript
function UserProfile() {
  const [user, setUser] = useState({ name: 'John', age: 30 });

  // ❌ NEVER DO THIS - Directly mutating state
  const handleChangeName = (newName) => {
    user.name = newName; // WRONG! Mutating existing object
    setUser(user); // React won't detect change (same reference)
  };

  return <div>{user.name}</div>;
}
```

**✅ RIGHT - Creating new object:**
```javascript
function UserProfile() {
  const [user, setUser] = useState({ name: 'John', age: 30, email: 'john@example.com' });

  // ✅ Correct way 1: Spread operator
  const handleChangeName = (newName) => {
    setUser({
      ...user,        // Spread existing properties
      name: newName   // Override only what changed
    });
  };

  // ✅ Correct way 2: Object.assign
  const handleChangeAge = (newAge) => {
    setUser(Object.assign({}, user, { age: newAge }));
  };

  // ✅ Correct way 3: Functional update (for derived state)
  const handleAddYear = () => {
    setUser(prevUser => ({
      ...prevUser,
      age: prevUser.age + 1
    }));
  };

  return (
    <div>
      <p>Name: {user.name}</p>
      <p>Age: {user.age}</p>
      <button onClick={() => handleChangeName('Jane')}>Change Name</button>
      <button onClick={() => handleChangeAge(31)}>Change Age</button>
      <button onClick={handleAddYear}>Add Year</button>
    </div>
  );
}
```

### Example 3: Array State (Immutability Pattern)

```javascript
function TodoList() {
  const [todos, setTodos] = useState([
    { id: 1, text: 'Learn React', completed: false },
    { id: 2, text: 'Learn Hooks', completed: false }
  ]);

  // ✅ Adding an item
  const addTodo = (text) => {
    const newTodo = {
      id: Date.now(), // Simple unique ID
      text: text,
      completed: false
    };
    setTodos([...todos, newTodo]); // Spread existing + add new
  };

  // ✅ Removing an item
  const removeTodo = (id) => {
    setTodos(todos.filter(todo => todo.id !== id));
  };

  // ✅ Updating an item
  const toggleTodo = (id) => {
    setTodos(todos.map(todo =>
      todo.id === id 
        ? { ...todo, completed: !todo.completed } 
        : todo
    ));
  };

  // ✅ Updating multiple items
  const clearCompleted = () => {
    setTodos(todos.filter(todo => !todo.completed));
  };

  return (
    <div>
      {todos.map(todo => (
        <div key={todo.id}>
          <input
            type="checkbox"
            checked={todo.completed}
            onChange={() => toggleTodo(todo.id)}
          />
          <span style={{ textDecoration: todo.completed ? 'line-through' : 'none' }}>
            {todo.text}
          </span>
          <button onClick={() => removeTodo(todo.id)}>Delete</button>
        </div>
      ))}
      <button onClick={clearCompleted}>Clear Completed</button>
    </div>
  );
}
```

### State Batching & Async Updates

**Important:** React batches state updates for performance.

```javascript
function BatchingExample() {
  const [count, setCount] = useState(0);
  const [name, setName] = useState('');

  const handleClick = () => {
    // These updates are BATCHED together
    // React will only re-render ONCE
    setCount(count + 1);
    setName('Updated');
    console.log('After setState calls');
    // This console.log runs BEFORE state updates!
  };

  // ✅ If you need to run code AFTER state updates, use useEffect
  useEffect(() => {
    console.log('Count changed to:', count);
  }, [count]);

  return (
    <div>
      <p>Count: {count}, Name: {name}</p>
      <button onClick={handleClick}>Update Both</button>
    </div>
  );
}
```

### Lazy Initial State (Performance Optimization)

```javascript
// ❌ EXPENSIVE COMPUTATION EVERY RENDER
function ExpensiveComponent() {
  const [data, setData] = useState(expensiveComputation());
  // expensiveComputation() runs on EVERY render!
}

// ✅ LAZY INITIALIZATION (Runs only ONCE on mount)
function OptimizedComponent() {
  const [data, setData] = useState(() => {
    console.log('Computing expensive value');
    return expensiveComputation();
  });
  // This runs only when component first mounts!
}

function expensiveComputation() {
  let sum = 0;
  for (let i = 0; i < 1000000; i++) {
    sum += i;
  }
  return sum;
}
```

### Functional Updates Pattern

```javascript
function Counter() {
  const [count, setCount] = useState(0);

  // ❌ Problem: Stale closure in rapid updates
  const handleRapidClick = () => {
    setCount(count + 1); // Uses current count
    setCount(count + 1); // Also uses SAME count (batched)
    // Result: Count increases by 1, not 2!
  };

  // ✅ Solution: Functional update
  const handleRapidClickFixed = () => {
    setCount(prev => prev + 1); // Gets actual latest value
    setCount(prev => prev + 1); // Builds on previous update
    // Result: Count increases by 2!
  };

  return (
    <div>
      <p>Count: {count}</p>
      <button onClick={handleRapidClick}>❌ Rapid Click (Wrong)</button>
      <button onClick={handleRapidClickFixed}>✅ Rapid Click (Fixed)</button>
    </div>
  );
}
```

---

## useEffect Hook - Complete Lifecycle

### Understanding Lifecycles in Functional Components

In class components, lifecycle was split across three methods. `useEffect` combines all of it in one unified pattern:

```javascript
useEffect(() => {
  // Setup code (equivalent to componentDidMount + componentDidUpdate)
  console.log('Effect running');

  return () => {
    // Cleanup code (equivalent to componentWillUnmount)
    console.log('Cleanup running');
  };
}, [dependencies]); // Dependency array
```

### Effect Lifecycle Mapping

```javascript
import { useEffect, useState } from 'react';

function EffectLifecycleDemo() {
  const [count, setCount] = useState(0);
  const [name, setName] = useState('');

  // ✅ RUN ONCE ON MOUNT (equivalent to componentDidMount)
  useEffect(() => {
    console.log('Component mounted!');
    console.log('Fetch initial data, subscribe to services, etc.');
    
    return () => {
      console.log('Cleanup on unmount');
    };
  }, []); // Empty dependency array = run only once on mount

  // ✅ RUN EVERY RENDER (equivalent to componentDidUpdate without condition)
  useEffect(() => {
    console.log('This runs after EVERY render');
    document.title = `Count: ${count}`;
  }); // No dependency array = run on every render

  // ✅ RUN WHEN SPECIFIC VALUES CHANGE (equivalent to conditional componentDidUpdate)
  useEffect(() => {
    console.log('Count changed to:', count);
  }, [count]); // Only re-run when count changes

  // ✅ RUN WHEN MULTIPLE VALUES CHANGE
  useEffect(() => {
    console.log(`Name changed to: ${name}`);
    console.log(`Count is now: ${count}`);
  }, [name, count]); // Re-run when EITHER name or count changes

  return (
    <div>
      <p>Count: {count}</p>
      <p>Name: {name}</p>
      <button onClick={() => setCount(count + 1)}>Increment</button>
      <input
        value={name}
        onChange={(e) => setName(e.target.value)}
        placeholder="Enter name"
      />
    </div>
  );
}
```

### Real-World Example: Data Fetching

```javascript
import { useEffect, useState } from 'react';

function UserData({ userId }) {
  const [user, setUser] = useState(null);
  const [loading, setLoading] = useState(true);
  const [error, setError] = useState(null);

  useEffect(() => {
    // 1. Start loading
    setLoading(true);
    setError(null);

    // 2. Fetch data
    fetch(`https://api.example.com/users/${userId}`)
      .then(response => {
        if (!response.ok) throw new Error('Failed to fetch');
        return response.json();
      })
      .then(data => {
        setUser(data);
        setLoading(false);
      })
      .catch(err => {
        setError(err.message);
        setLoading(false);
      });

    // 3. Cleanup (abort previous request if component unmounts)
    return () => {
      console.log('Cleaning up fetch for userId:', userId);
    };
  }, [userId]); // Re-fetch when userId changes

  if (loading) return <p>Loading...</p>;
  if (error) return <p>Error: {error}</p>;
  if (!user) return <p>No user found</p>;

  return (
    <div>
      <h2>{user.name}</h2>
      <p>Email: {user.email}</p>
    </div>
  );
}
```

### Cleanup Functions (Critical for Production)

Cleanup functions prevent memory leaks and bugs:

```javascript
import { useEffect, useState } from 'react';

// ❌ MEMORY LEAK - No cleanup
function BadTimer() {
  const [count, setCount] = useState(0);

  useEffect(() => {
    const interval = setInterval(() => {
      setCount(c => c + 1);
    }, 1000);
    // Interval keeps running even after component unmounts!
  }, []);

  return <p>Count: {count}</p>;
}

// ✅ CORRECT - With cleanup
function GoodTimer() {
  const [count, setCount] = useState(0);

  useEffect(() => {
    const interval = setInterval(() => {
      setCount(c => c + 1);
    }, 1000);

    // Cleanup function clears interval when component unmounts
    return () => {
      clearInterval(interval);
    };
  }, []);

  return <p>Count: {count}</p>;
}

// ❌ MEMORY LEAK - Event listener not removed
function BadEventListener() {
  useEffect(() => {
    const handleResize = () => console.log('Window resized');
    window.addEventListener('resize', handleResize);
    // Listener stays attached even after unmount!
  }, []);
}

// ✅ CORRECT - Remove event listener
function GoodEventListener() {
  useEffect(() => {
    const handleResize = () => console.log('Window resized');
    window.addEventListener('resize', handleResize);

    return () => {
      window.removeEventListener('resize', handleResize);
    };
  }, []);
}

// ❌ MEMORY LEAK - Subscription not unsubscribed
function BadSubscription() {
  useEffect(() => {
    const subscription = myObservable.subscribe(value => {
      console.log(value);
    });
    // Subscription stays active forever!
  }, []);
}

// ✅ CORRECT - Unsubscribe
function GoodSubscription() {
  useEffect(() => {
    const subscription = myObservable.subscribe(value => {
      console.log(value);
    });

    return () => {
      subscription.unsubscribe();
    };
  }, []);
}
```

### Dependency Array Gotchas (The #1 Source of Bugs)

```javascript
import { useEffect, useState } from 'react';

// ❌ INFINITE LOOP - Missing dependency array
function BadInfiniteLoop() {
  const [count, setCount] = useState(0);

  useEffect(() => {
    setCount(count + 1);
    // No dependency array = runs after every render
    // Effect updates state = triggers new render
    // New render triggers effect again = INFINITE LOOP!
  });

  return <p>Count: {count}</p>;
}

// ✅ CORRECT - With dependency array
function FixedNoInfiniteLoop() {
  const [count, setCount] = useState(0);

  useEffect(() => {
    const timer = setTimeout(() => {
      setCount(c => c + 1);
    }, 1000);

    return () => clearTimeout(timer);
  }, []); // Empty array = run once, never again
}

// ❌ STALE CLOSURE PROBLEM
function BadStalelosure() {
  const [count, setCount] = useState(0);

  useEffect(() => {
    const interval = setInterval(() => {
      console.log(`Count is: ${count}`); // Always logs 0!
    }, 1000);

    return () => clearInterval(interval);
  }, []); // Missing count in dependencies!
  // count never changes in the interval callback
}

// ✅ CORRECT - Include all dependencies
function GoodClosure() {
  const [count, setCount] = useState(0);

  useEffect(() => {
    const interval = setInterval(() => {
      console.log(`Count is: ${count}`); // Logs current count
    }, 1000);

    return () => clearInterval(interval);
  }, [count]); // Include count as dependency
  // Effect re-runs whenever count changes
  // New interval created with updated count
}

// ⚠️ CONDITIONAL EFFECTS (Advanced pattern)
function ConditionalEffect({ shouldRun }) {
  const [data, setData] = useState(null);

  useEffect(() => {
    if (!shouldRun) return; // Early return is OK

    // Fetch data only when shouldRun is true
    fetch('/api/data')
      .then(r => r.json())
      .then(setData);
  }, [shouldRun]);

  return <div>{data ? 'Loaded' : 'Not loaded'}</div>;
}
```

### Multiple Effects (Separate Concerns)

```javascript
function ComponentWithMultipleEffects({ userId }) {
  const [user, setUser] = useState(null);
  const [posts, setPosts] = useState([]);
  const [pageTitle, setPageTitle] = useState('');

  // Effect 1: Fetch user data when userId changes
  useEffect(() => {
    fetch(`/api/users/${userId}`)
      .then(r => r.json())
      .then(setUser);
  }, [userId]);

  // Effect 2: Fetch posts whenever user changes
  useEffect(() => {
    if (!user) return;
    
    fetch(`/api/users/${user.id}/posts`)
      .then(r => r.json())
      .then(setPosts);
  }, [user?.id]); // Use optional chaining to be safe

  // Effect 3: Update page title when user loads
  useEffect(() => {
    if (user) {
      document.title = `${user.name}'s Profile`;
    }
  }, [user?.name]);

  // Effect 4: Log whenever posts change
  useEffect(() => {
    console.log(`User has ${posts.length} posts`);
  }, [posts.length]);

  return (
    <div>
      {user && <h1>{user.name}</h1>}
      <p>{posts.length} posts</p>
    </div>
  );
}
```

---

## Common Mistakes & Solutions

### Mistake 1: Forgetting Cleanup Functions

**Problem:**
```javascript
useEffect(() => {
  const subscription = DataStore.subscribe(user => setUser(user));
  // Subscription never cancelled = memory leak!
}, []);
```

**Solution:**
```javascript
useEffect(() => {
  const subscription = DataStore.subscribe(user => setUser(user));
  return () => subscription.unsubscribe(); // Cleanup!
}, []);
```

### Mistake 2: Overusing State

**Problem:**
```javascript
function Form() {
  const [name, setName] = useState('');
  const [email, setEmail] = useState('');
  const [age, setAge] = useState('');
  const [phone, setPhone] = useState('');
  // Too many state variables!
}
```

**Solution:**
```javascript
function Form() {
  // Group related data together
  const [formData, setFormData] = useState({
    name: '',
    email: '',
    age: '',
    phone: ''
  });

  const handleChange = (field, value) => {
    setFormData(prev => ({
      ...prev,
      [field]: value
    }));
  };

  return (
    <>
      <input onChange={(e) => handleChange('name', e.target.value)} />
      <input onChange={(e) => handleChange('email', e.target.value)} />
    </>
  );
}
```

### Mistake 3: Effect Misuse (Wrong Pattern)

**Problem:**
```javascript
useEffect(() => {
  // This runs after component renders, then causes re-render
  // Can lead to performance issues
  updateSomeGlobalState();
}, []); // Runs only once... supposedly
```

**Solution:**
```javascript
// Move this to event handler instead
function MyComponent() {
  const handleClick = () => {
    updateSomeGlobalState(); // Triggered by action, not by effect
  };

  return <button onClick={handleClick}>Click me</button>;
}
```

---

## Practical Examples & Patterns

### Pattern 1: Form Validation

```javascript
import { useEffect, useState } from 'react';

function RegistrationForm() {
  const [email, setEmail] = useState('');
  const [password, setPassword] = useState('');
  const [errors, setErrors] = useState({});
  const [touched, setTouched] = useState({});

  // Validate whenever email or password changes
  useEffect(() => {
    const newErrors = {};

    if (touched.email && !email.includes('@')) {
      newErrors.email = 'Invalid email';
    }

    if (touched.password && password.length < 8) {
      newErrors.password = 'Password must be 8+ characters';
    }

    setErrors(newErrors);
  }, [email, password, touched]);

  const handleSubmit = (e) => {
    e.preventDefault();
    if (Object.keys(errors).length === 0) {
      console.log('Form submitted:', { email, password });
    }
  };

  return (
    <form onSubmit={handleSubmit}>
      <input
        value={email}
        onChange={(e) => setEmail(e.target.value)}
        onBlur={() => setTouched(prev => ({ ...prev, email: true }))}
        placeholder="Email"
      />
      {errors.email && <span className="error">{errors.email}</span>}

      <input
        value={password}
        onChange={(e) => setPassword(e.target.value)}
        onBlur={() => setTouched(prev => ({ ...prev, password: true }))}
        type="password"
        placeholder="Password"
      />
      {errors.password && <span className="error">{errors.password}</span>}

      <button type="submit" disabled={Object.keys(errors).length > 0}>
        Register
      </button>
    </form>
  );
}
```

### Pattern 2: Debounced Search

```javascript
import { useEffect, useState } from 'react';

function SearchUsers() {
  const [query, setQuery] = useState('');
  const [results, setResults] = useState([]);
  const [loading, setLoading] = useState(false);

  useEffect(() => {
    if (!query.trim()) {
      setResults([]);
      return;
    }

    // Debounce: wait 500ms before searching
    const timer = setTimeout(() => {
      setLoading(true);
      fetch(`/api/users/search?q=${encodeURIComponent(query)}`)
        .then(r => r.json())
        .then(data => {
          setResults(data);
          setLoading(false);
        });
    }, 500);

    // Cleanup: clear timer if component unmounts or query changes
    return () => clearTimeout(timer);
  }, [query]); // Re-run when query changes

  return (
    <div>
      <input
        value={query}
        onChange={(e) => setQuery(e.target.value)}
        placeholder="Search..."
      />
      {loading && <p>Loading...</p>}
      <ul>
        {results.map(user => (
          <li key={user.id}>{user.name}</li>
        ))}
      </ul>
    </div>
  );
}
```

### Pattern 3: Window Resize Handler

```javascript
function ResponsiveComponent() {
  const [windowSize, setWindowSize] = useState({
    width: window.innerWidth,
    height: window.innerHeight
  });

```javascript
function ResponsiveComponent() {
  const [windowSize, setWindowSize] = useState({
    width: window.innerWidth,
    height: window.innerHeight
  });

  useEffect(() => {
    const handleResize = () => {
      setWindowSize({
        width: window.innerWidth,
        height: window.innerHeight
      });
    };

    // Add listener
    window.addEventListener('resize', handleResize);

    // Cleanup: remove listener
    return () => {
      window.removeEventListener('resize', handleResize);
    };
  }, []);

  return (
    <div>
      <p>Width: {windowSize.width}px</p>
      <p>Height: {windowSize.height}px</p>
    </div>
  );
}
```

---

## Exercises & Assessment

### Exercise 1: Simple Counter (Difficulty: Easy)
Create a component with:
- Counter starting at 0
- Increment button (adds 1)
- Decrement button (subtracts 1)
- Reset button (resets to 0)

**Hint:** Use single useState for count.

### Exercise 2: Toggle Visibility (Difficulty: Easy)
Create a component with:
- Toggle button that shows/hides content
- Content that displays/hides based on state

**Hint:** Use boolean state.

### Exercise 3: User Form with Validation (Difficulty: Medium)
Create a registration form with:
- Name, email, password, and confirm password fields
- Real-time validation
- Submit button that's disabled until all fields are valid
- Display error messages

**Hint:** Use useEffect to validate whenever inputs change.

### Exercise 4: API Integration (Difficulty: Medium)
Create a component that:
- Fetches user data from JSONPlaceholder API (`https://jsonplaceholder.typicode.com/users/1`)
- Shows loading state while fetching
- Shows error message if fetch fails
- Displays user data when loaded

**Hint:** Use useEffect with empty dependency array to fetch on mount.

### Exercise 5: Todo App (Difficulty: Hard)
Create a complete todo app with:
- Add new todos
- Mark todos as complete/incomplete
- Delete todos
- Show count of completed vs pending todos
- Persist todos to localStorage (use useEffect with cleanup)

**Hint:** Use useEffect to save todos whenever they change.

---

## Key Takeaways

✅ **Always clean up** in useEffect (subscriptions, timers, listeners)
✅ **Never mutate state** - create new objects/arrays
✅ **Use dependency array** correctly - include all values the effect depends on
✅ **Prefer multiple effects** over one giant effect
✅ **Use functional updates** for state that depends on previous state
✅ **Lazy initialize** expensive computations
✅ **Handle loading and error states** in data fetching

---

*Last Updated: January 2026 | React 18+ | For Production Use*
